In [12]:
import pandas as pd

input_df = pd.read_csv('addresses.csv', index_col=0)
input_df.head()

,0
0,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja
1,3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR
2,bc1q0xur68lq587lw8aqf96f4qqcmum6sqcpyyy94a
3,3JjtnDXknvY7HWidn5wt5BdZsRdXCGEt8y
4,3J9rQSzvNThPdN5FDbJAD3N8Wdce1KF6Ms


In [14]:
btc_hashes = input_df['0'].values
URL = "https://blockstream.info/api/address/{}/txs/chain"

for btc_hash in btc_hashes:
    